# IBM Data Science Capstone Project

Lets import & install the necessary Libraries

In [2]:
!conda install -c conda-forge geopy -y
!pip install geocoder
!pip install folium
!pip install beautifulsoup4
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("All Required Libraries Imported!")

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    openssl-1.1.1j             |       h7f98852_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data,"html5lib")

In [4]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

content = wiki_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n').replace(']','')
            i = i + 1
        elif i == 1:
            borough = td.text.strip('\n').replace(']','')
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)
toronto.head()

,Postalcode,Borough,Neighborhood
0,0,0,0
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [5]:
df = toronto.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]
df=df[1:]
df=df.reset_index(drop=True)

In [6]:
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
df.shape

(103, 3)

In [8]:
df.describe()

,Postalcode,Borough,Neighborhood
count,103,103,103
unique,103,11,99
top,M4G,North York,Downsview
freq,1,24,4


In [9]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.709020000000066, -79.36348999999996]

In [10]:
# Retrieving Postal Code Co-ordinates
postal_codes = df['Postalcode']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

# Adding Columns Latitude & Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


In [17]:
df

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.66263,-79.52831
6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
7,M3B,North York,Don Mills,43.74923,-79.36186
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804


# Map of Toronto & Neighborhoods

In [15]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let's simplify the above map and segment and cluster only the neighborhoods in Etobicoke. So let's slice the original dataframe and create a new dataframe of the Etobicoke data.

In [21]:
etobicoke_data = df[df['Borough'] == 'Etobicoke'].reset_index(drop=True)
etobicoke_data

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.66263,-79.52831
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.65034,-79.55362
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.64857,-79.57825
3,M9P,Etobicoke,Westmount,43.69630,-79.52926
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.68681,-79.55728
5,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.60987,-79.49809
6,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.74453,-79.58624
7,M8W,Etobicoke,"Alderwood, Long Branch",43.60124,-79.53879
8,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.71174,-79.57941
9,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113


Now lets get the geographic co-ordinates of Etobicoke

In [22]:
address = 'Etobicoke, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke are 43.6435559, -79.5656326.


Let's visualize Etobicoke & the neighborhoods in it.

In [23]:
# create map of Etobicoke using latitude and longitude values
map_etobicoke = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etobicoke)  
    
map_etobicoke

Define Foursquare Credentials and Version

In [24]:
CLIENT_ID = 'DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY' # your Foursquare ID
CLIENT_SECRET = '1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY
CLIENT_SECRET:1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B


Let's explore the first neighborhood in our dataframe.

In [25]:
etobicoke_data.loc[0, 'Neighborhood']

'Islington Avenue, Humber Valley Village'

Get the neighborhood's latitude and longitude values.

In [26]:
neighborhood_latitude = etobicoke_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = etobicoke_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = etobicoke_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Islington Avenue, Humber Valley Village are 43.662630000000036, -79.52830999999998.


Now, let's get the top 100 venues that are in Avenue, Humber Valley Village within a radius of 500 meters.

In [28]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60486e9b10a60373a615f288'},
 'response': {'headerLocation': 'Edenbridge - Humber Valley',
  'headerFullLocation': 'Edenbridge - Humber Valley, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 43.66713000450004,
    'lng': -79.5221011346927},
   'sw': {'lat': 43.65812999550003, 'lng': -79.53451886530725}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bfd53764cf820a13849ecf4',
       'name': "Java Joe's Village Cafe",
       'location': {'address': '1500 Islington Ave',
        'crossStreet': 'at Rathburn Rd',
        'lat': 43.662460906352436,
        'lng': -79.53205381416235,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.662460906352436,
      

In [29]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now lets clean the json and structure it into a pandas dataframe.

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Java Joe's Village Cafe,Café,43.662461,-79.532054
1,TD Canada Trust,Bank,43.662545,-79.531749
2,Shoppers Drug Mart,Pharmacy,43.663067,-79.531753
3,Thorncrest Drug Store,Pharmacy,43.662988,-79.531817
4,Foodland - Toronto,Grocery Store,43.662724,-79.531984


In [33]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


Let's create a function to repeat the same process to all the neighborhoods in Etobicoke

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
etobicoke_venues = getNearbyVenues(names=etobicoke_data['Neighborhood'],
                                   latitudes=etobicoke_data['Latitude'],
                                   longitudes=etobicoke_data['Longitude']
                                  )

Islington Avenue, Humber Valley Village
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Westmount
Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens
New Toronto, Mimico South, Humber Bay Shores
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
Alderwood, Long Branch
Northwest, West Humber - Clairville
The Kingsway, Montgomery Road, Old Mill North
Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East
Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West


In [38]:
print(etobicoke_venues.shape)
etobicoke_venues.head()

(80, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Islington Avenue, Humber Valley Village",43.66263,-79.52831,Java Joe's Village Cafe,43.662461,-79.532054,Café
1,"Islington Avenue, Humber Valley Village",43.66263,-79.52831,TD Canada Trust,43.662545,-79.531749,Bank
2,"Islington Avenue, Humber Valley Village",43.66263,-79.52831,Shoppers Drug Mart,43.663067,-79.531753,Pharmacy
3,"Islington Avenue, Humber Valley Village",43.66263,-79.52831,Thorncrest Drug Store,43.662988,-79.531817,Pharmacy
4,"Islington Avenue, Humber Valley Village",43.66263,-79.52831,Foodland - Toronto,43.662724,-79.531984,Grocery Store


In [39]:
etobicoke_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alderwood, Long Branch",4,4,4,4,4,4
"Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood",6,6,6,6,6,6
"Islington Avenue, Humber Valley Village",8,8,8,8,8,8
"Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens",5,5,5,5,5,5
"Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West",11,11,11,11,11,11
"New Toronto, Mimico South, Humber Bay Shores",5,5,5,5,5,5
"Northwest, West Humber - Clairville",7,7,7,7,7,7
"Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East",7,7,7,7,7,7
"South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens",15,15,15,15,15,15


In [40]:
print('There are {} uniques categories.'.format(len(etobicoke_venues['Venue Category'].unique())))

There are 49 uniques categories.


# Analyze Each Neighborhood

In [41]:
# one hot encoding
etobicoke_onehot = pd.get_dummies(etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
etobicoke_onehot['Neighborhood'] = etobicoke_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [etobicoke_onehot.columns[-1]] + list(etobicoke_onehot.columns[:-1])
etobicoke_onehot = etobicoke_onehot[fixed_columns]

etobicoke_onehot.head()

,Neighborhood,Arts & Crafts Store,Auto Garage,Bank,Beer Store,Burger Joint,Burrito Place,Bus Line,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Rec Center,Convenience Store,Dance Studio,Discount Store,Eastern European Restaurant,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Fried Chicken Joint,Gas Station,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Italian Restaurant,Japanese Restaurant,Mattress Store,Middle Eastern Restaurant,Miscellaneous Shop,Music Venue,Park,Pet Store,Pharmacy,Pizza Place,Pub,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Storage Facility,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Video Store,Yoga Studio
0,"Islington Avenue, Humber Valley Village",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Islington Avenue, Humber Valley Village",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Islington Avenue, Humber Valley Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Islington Avenue, Humber Valley Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Islington Avenue, Humber Valley Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
etobicoke_onehot.shape

(80, 50)

In [43]:
etobicoke_grouped = etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
etobicoke_grouped

,Neighborhood,Arts & Crafts Store,Auto Garage,Bank,Beer Store,Burger Joint,Burrito Place,Bus Line,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Rec Center,Convenience Store,Dance Studio,Discount Store,Eastern European Restaurant,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Fried Chicken Joint,Gas Station,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Italian Restaurant,Japanese Restaurant,Mattress Store,Middle Eastern Restaurant,Miscellaneous Shop,Music Venue,Park,Pet Store,Pharmacy,Pizza Place,Pub,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Storage Facility,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Video Store,Yoga Studio
0,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.166667,0.00,0.00,0.000000,0.000000,0.166667,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.166667,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,"Islington Avenue, Humber Valley Village",0.0,0.000000,0.125000,0.000000,0.000000,0.000000,0.0,0.125,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.125000,0.000000,0.250000,0.000000,0.00,0.000000,0.000000,0.125000,0.125,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
3,"Kingsview Village, St. Phillips, Martin Grove ...",0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000,0.000000,0.000000,0.2,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.200000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
4,"Mimico NW, The Queensway West, South of Bloor,...",0.0,0.000000,0.090909,0.000000,0.090909,0.090909,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.090909,0.000000,0.000000,0.090909,0.000000,0.090909,0.0,0.000000,0.090909,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.090909,0.000000,0.0,0.090909,0.000000,0.0
5,"New Toronto, Mimico South, Humber Bay Shores",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.200000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.200,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.2
6,"Northwest, West Humber - Clairville",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.142857,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.142857,0.142857,0.000000,0.000,0.142857,0.000000,0.000000,0.0,0.000000,0.000000,0.0
7,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0.0,0.000000,0.142857,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.142857,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.1428

In [44]:
etobicoke_grouped.shape

(11, 50)

Let's print each neighborhood along with the top 5 most common venues

In [45]:
num_top_venues = 5

for hood in etobicoke_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = etobicoke_grouped[etobicoke_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood, Long Branch----
               venue  freq
0  Convenience Store  0.25
1                Pub  0.25
2       Dance Studio  0.25
3                Gym  0.25
4       Tennis Court  0.00


----Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood----
               venue  freq
0      Grocery Store  0.17
1  Fish & Chips Shop  0.17
2               Park  0.17
3      Shopping Mall  0.17
4  Electronics Store  0.17


----Islington Avenue, Humber Valley Village----
           venue  freq
0       Pharmacy  0.25
1  Grocery Store  0.12
2           Bank  0.12
3           Café  0.12
4           Park  0.12


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                 venue  freq
0  Arts & Crafts Store   0.2
1             Bus Line   0.2
2          Music Venue   0.2
3       Clothing Store   0.2
4          Pizza Place   0.2


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West----
                        

Write a function to sort the venues in descending order.

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Let's create the new dataframe and display the top 10 venues for each neighborhood.

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = etobicoke_grouped['Neighborhood']

for ind in np.arange(etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alderwood, Long Branch",Gym,Dance Studio,Convenience Store,Pub,Yoga Studio,Coffee Shop,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Electronics Store
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Grocery Store,Fish & Chips Shop,Electronics Store,Shopping Mall,Park,College Rec Center,Coffee Shop,Flower Shop,Fast Food Restaurant,Eastern European Restaurant
2,"Islington Avenue, Humber Valley Village",Pharmacy,Skating Rink,Park,Café,Shopping Mall,Grocery Store,Bank,Burrito Place,College Rec Center,Fish & Chips Shop
3,"Kingsview Village, St. Phillips, Martin Grove ...",Arts & Crafts Store,Bus Line,Music Venue,Clothing Store,Pizza Place,Burger Joint,Burrito Place,Flower Shop,Fish & Chips Shop,Fast Food Restaurant
4,"Mimico NW, The Queensway West, South of Bloor,...",Pet Store,Miscellaneous Shop,Thai Restaurant,Bank,Sushi Restaurant,Burger Joint,Burrito Place,Eastern European Restaurant,Gym,Gym / Fitness Center


# Clustering Neighborhoods

In [56]:
# set number of clusters
kclusters = 5

etobicoke_grouped_clustering = etobicoke_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 2, 2, 4, 1, 2, 1, 1, 1, 0], dtype=int32)

Now let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [62]:
etobicoke_merged = etobicoke_data

# merge etobicoke_grouped with etobicoke_data to add latitude/longitude for each neighborhood
etobicoke_merged = etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
etobicoke_merged=etobicoke_merged.dropna()
etobicoke_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.66263,-79.52831,2.0,Pharmacy,Skating Rink,Park,Café,Shopping Mall,Grocery Store,Bank,Burrito Place,College Rec Center,Fish & Chips Shop
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.65034,-79.55362,0.0,Pizza Place,Chinese Restaurant,Tea Room,Sandwich Place,Gift Shop,Beer Store,College Rec Center,Flower Shop,Fish & Chips Shop,Fast Food Restaurant
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.64857,-79.57825,2.0,Grocery Store,Fish & Chips Shop,Electronics Store,Shopping Mall,Park,College Rec Center,Coffee Shop,Flower Shop,Fast Food Restaurant,Eastern European Restaurant
3,M9P,Etobicoke,Westmount,43.69630,-79.52926,0.0,Pizza Place,Middle Eastern Restaurant,Sandwich Place,Chinese Restaurant,Coffee Shop,Yoga Studio,College Rec Center,Flower Shop,Fish & Chips Shop,Fast Food Restaurant
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.68681,-79.55728,4.0,Arts & Crafts Store,Bus Line,Music Venue,Clothing Store,Pizza Place,Burger Joint,Burrito Place,Flower Shop,Fish & Chips Shop,Fast Food Restaurant


Finally, let's visualize the resulting clusters

In [63]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(etobicoke_merged['Latitude'], etobicoke_merged['Longitude'], etobicoke_merged['Neighborhood'], etobicoke_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examining Clusters

### Cluster # 1

In [64]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 0, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Etobicoke,0.0,Pizza Place,Chinese Restaurant,Tea Room,Sandwich Place,Gift Shop,Beer Store,College Rec Center,Flower Shop,Fish & Chips Shop,Fast Food Restaurant
3,Etobicoke,0.0,Pizza Place,Middle Eastern Restaurant,Sandwich Place,Chinese Restaurant,Coffee Shop,Yoga Studio,College Rec Center,Flower Shop,Fish & Chips Shop,Fast Food Restaurant


### Cluster # 2

In [65]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 1, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Etobicoke,1.0,Grocery Store,Caribbean Restaurant,Fast Food Restaurant,Video Store,Discount Store,Japanese Restaurant,Coffee Shop,Park,Pharmacy,Pizza Place
8,Etobicoke,1.0,Gas Station,Coffee Shop,Gym,Storage Facility,Sandwich Place,Restaurant,Middle Eastern Restaurant,Fish & Chips Shop,Fast Food Restaurant,Electronics Store
10,Etobicoke,1.0,Coffee Shop,Flower Shop,Bank,Sushi Restaurant,Fast Food Restaurant,Italian Restaurant,Park,College Rec Center,Fish & Chips Shop,Electronics Store
11,Etobicoke,1.0,Pet Store,Miscellaneous Shop,Thai Restaurant,Bank,Sushi Restaurant,Burger Joint,Burrito Place,Eastern European Restaurant,Gym,Gym / Fitness Center


### Cluster # 3

In [66]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 2, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Etobicoke,2.0,Pharmacy,Skating Rink,Park,Café,Shopping Mall,Grocery Store,Bank,Burrito Place,College Rec Center,Fish & Chips Shop
2,Etobicoke,2.0,Grocery Store,Fish & Chips Shop,Electronics Store,Shopping Mall,Park,College Rec Center,Coffee Shop,Flower Shop,Fast Food Restaurant,Eastern European Restaurant
5,Etobicoke,2.0,Yoga Studio,Skating Rink,Park,Grocery Store,Tennis Court,Tea Room,Clothing Store,Fish & Chips Shop,Fast Food Restaurant,Electronics Store


### Cluster # 4

In [67]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 3, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Etobicoke,3.0,Gym,Dance Studio,Convenience Store,Pub,Yoga Studio,Coffee Shop,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Electronics Store


### Cluster # 5

In [68]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 4, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Etobicoke,4.0,Arts & Crafts Store,Bus Line,Music Venue,Clothing Store,Pizza Place,Burger Joint,Burrito Place,Flower Shop,Fish & Chips Shop,Fast Food Restaurant
